In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import LinearSVC, SVC, SVR

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import metrics

In [17]:
df = pd.read_csv('./data/bos_cleaned_allcolumns.csv')
df.set_index('id', inplace=True)

pd.options.display.max_columns = 100
pd.options.display.max_rows = 3000
pd.options.display.max_colwidth = 200

In [18]:
df.tail()

,name,host_is_superhost,host_identity_verified,nb,property,room,accommodates,bedrooms,beds,amenities,price,dataset,superhost,id_verified,nb_Back Bay,nb_Bay Village,nb_Beacon Hill,nb_Brighton,nb_Charlestown,nb_Chinatown,nb_Dorchester,nb_Downtown,nb_East Boston,nb_Fenway,nb_Hyde Park,nb_Jamaica Plain,nb_Leather District,nb_Longwood Medical Area,nb_Mattapan,nb_Mission Hill,nb_North End,nb_Roslindale,nb_Roxbury,nb_South Boston,nb_South Boston Waterfront,nb_South End,nb_West End,nb_West Roxbury,room_Hotel room,room_Private room,room_Shared room,property_Apartment,property_Barn,property_Bed and breakfast,property_Boat,property_Boutique hotel,property_Bungalow,property_Castle,property_Condominium,property_Cottage,property_Entire apartment,property_Entire bed and breakfast,property_Entire condominium,property_Entire cottage,property_Entire guest suite,property_Entire guesthouse,property_Entire home/apt,property_Entire house,property_Entire loft,property_Entire place,property_Entire serviced apartment,property_Entire townhouse,property_Guest suite,property_Guesthouse,property_Hostel,property_Hotel,property_House,property_Houseboat,property_Loft,property_Other,property_Private room in apartment,property_Private room in bed and breakfast,property_Private room in bungalow,property_Private room in castle,property_Private room in condominium,property_Private room in guest suite,property_Private room in guesthouse,property_Private room in house,property_Private room in loft,property_Private room in townhouse,property_Private room in villa,property_Room in aparthotel,property_Room in bed and breakfast,property_Room in boutique hotel,property_Room in hostel,property_Room in hotel,property_Serviced apartment,property_Shared room in apartment,property_Shared room in bed and breakfast,property_Shared room in boutique hotel,property_Shared room in condominium,property_Shared room in house,property_Shared room in townhouse,property_Townhouse,property_Villa
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
46020985,Stunning 2BR in Downtown Boston | Evonify,f,t,Downtown,Entire apartment,Entire home/apt,6,2.0,2.0,"[""Shower gel"", ""Body soap"", ""Shampoo"", ""Smoke alarm"", ""TV"", ""Bed linens"", ""Paid parking garage off premises"", ""Private entrance"", ""First aid kit"", ""Hangers"", ""Carbon monoxide alarm"", ""Freezer"", ""I...",319.0,post-covid,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
46021420,Stunning 1BR in Downtown + 100 WalkScore | Evonify,f,t,Beacon Hill,Entire apartment,Entire home/apt,4,1.0,1.0,"[""Shower gel"", ""Shampoo"", ""Smoke alarm"", ""TV"", ""Bed linens"", ""Private entrance"", ""Keypad"", ""First aid kit"", ""Hangers"", ""Carbon monoxide alarm"", ""Iron"", ""Extra pillows and blankets"", ""Wifi"", ""Oven""...",239.0,post-covid,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
46021809,Spacious and Modern 2BD in the Heart of Boston,f,t,Roxbury,Entire apartment,Entire home/apt,4,1.0,1.0,"[""Air conditioning"", ""Heating"", ""Laptop-friendly workspace"", ""TV"", ""Essentials"", ""Kitchen"", ""Wifi""]",47.0,post-covid,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
46022872,"Room in Large Brookline House, Phenomenal Location",f,t,Allston,Private room in house,Private room,1,1.0,1.0,"[""Hangers"", ""Heating"", ""Laptop-friendly workspace"", ""Smoke alarm"", ""Washer"", ""Kitchen"", ""Wifi"", ""Dryer"", ""Keypad""]",44.0,post-covid,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
46024344,"Furnished Room, Big Brookline

In [19]:
df = df[df['dataset'] == 'post-covid']

In [20]:
df.head(1)

,name,host_is_superhost,host_identity_verified,nb,property,room,accommodates,bedrooms,beds,amenities,price,dataset,superhost,id_verified,nb_Back Bay,nb_Bay Village,nb_Beacon Hill,nb_Brighton,nb_Charlestown,nb_Chinatown,nb_Dorchester,nb_Downtown,nb_East Boston,nb_Fenway,nb_Hyde Park,nb_Jamaica Plain,nb_Leather District,nb_Longwood Medical Area,nb_Mattapan,nb_Mission Hill,nb_North End,nb_Roslindale,nb_Roxbury,nb_South Boston,nb_South Boston Waterfront,nb_South End,nb_West End,nb_West Roxbury,room_Hotel room,room_Private room,room_Shared room,property_Apartment,property_Barn,property_Bed and breakfast,property_Boat,property_Boutique hotel,property_Bungalow,property_Castle,property_Condominium,property_Cottage,property_Entire apartment,property_Entire bed and breakfast,property_Entire condominium,property_Entire cottage,property_Entire guest suite,property_Entire guesthouse,property_Entire home/apt,property_Entire house,property_Entire loft,property_Entire place,property_Entire serviced apartment,property_Entire townhouse,property_Guest suite,property_Guesthouse,property_Hostel,property_Hotel,property_House,property_Houseboat,property_Loft,property_Other,property_Private room in apartment,property_Private room in bed and breakfast,property_Private room in bungalow,property_Private room in castle,property_Private room in condominium,property_Private room in guest suite,property_Private room in guesthouse,property_Private room in house,property_Private room in loft,property_Private room in townhouse,property_Private room in villa,property_Room in aparthotel,property_Room in bed and breakfast,property_Room in boutique hotel,property_Room in hostel,property_Room in hotel,property_Serviced apartment,property_Shared room in apartment,property_Shared room in bed and breakfast,property_Shared room in boutique hotel,property_Shared room in condominium,property_Shared room in house,property_Shared room in townhouse,property_Townhouse,property_Villa
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3781,HARBORSIDE-Walk to subway,f,f,East Boston,Entire apartment,Entire home/apt,2,1.0,0.0,"[""Cable TV"", ""Shampoo"", ""Smoke alarm"", ""TV"", ""Bed linens"", ""Free street parking"", ""Hangers"", ""Patio or balcony"", ""Carbon monoxide alarm"", ""Iron"", ""Extra pillows and blankets"", ""Wifi"", ""Oven"", ""Dry...",150.0,post-covid,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [21]:
df.shape

(2830, 95)

In [22]:
df.columns

Index(['name', 'host_is_superhost', 'host_identity_verified', 'nb', 'property',
       'room', 'accommodates', 'bedrooms', 'beds', 'amenities', 'price',
       'dataset', 'superhost', 'id_verified', 'nb_Back Bay', 'nb_Bay Village',
       'nb_Beacon Hill', 'nb_Brighton', 'nb_Charlestown', 'nb_Chinatown',
       'nb_Dorchester', 'nb_Downtown', 'nb_East Boston', 'nb_Fenway',
       'nb_Hyde Park', 'nb_Jamaica Plain', 'nb_Leather District',
       'nb_Longwood Medical Area', 'nb_Mattapan', 'nb_Mission Hill',
       'nb_North End', 'nb_Roslindale', 'nb_Roxbury', 'nb_South Boston',
       'nb_South Boston Waterfront', 'nb_South End', 'nb_West End',
       'nb_West Roxbury', 'room_Hotel room', 'room_Private room',
       'room_Shared room', 'property_Apartment', 'property_Barn',
       'property_Bed and breakfast', 'property_Boat',
       'property_Boutique hotel', 'property_Bungalow', 'property_Castle',
       'property_Condominium', 'property_Cottage', 'property_Entire apartment',
       '

In [23]:
features = list(df.columns.drop(['name', 'host_is_superhost', 'host_identity_verified', 'nb', 'property',
       'room', 'amenities', 'dataset', 'price']))

In [24]:
features

['accommodates',
 'bedrooms',
 'beds',
 'superhost',
 'id_verified',
 'nb_Back Bay',
 'nb_Bay Village',
 'nb_Beacon Hill',
 'nb_Brighton',
 'nb_Charlestown',
 'nb_Chinatown',
 'nb_Dorchester',
 'nb_Downtown',
 'nb_East Boston',
 'nb_Fenway',
 'nb_Hyde Park',
 'nb_Jamaica Plain',
 'nb_Leather District',
 'nb_Longwood Medical Area',
 'nb_Mattapan',
 'nb_Mission Hill',
 'nb_North End',
 'nb_Roslindale',
 'nb_Roxbury',
 'nb_South Boston',
 'nb_South Boston Waterfront',
 'nb_South End',
 'nb_West End',
 'nb_West Roxbury',
 'room_Hotel room',
 'room_Private room',
 'room_Shared room',
 'property_Apartment',
 'property_Barn',
 'property_Bed and breakfast',
 'property_Boat',
 'property_Boutique hotel',
 'property_Bungalow',
 'property_Castle',
 'property_Condominium',
 'property_Cottage',
 'property_Entire apartment',
 'property_Entire bed and breakfast',
 'property_Entire condominium',
 'property_Entire cottage',
 'property_Entire guest suite',
 'property_Entire guesthouse',
 'property_Entire

In [25]:
X = df[features]
y = df['price']

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

### Scale the data

In [27]:
sc = StandardScaler()

In [28]:
sc.fit(X_train)

StandardScaler()

In [29]:
X_train_sc = sc.transform(X_train)
X_test_sc = sc.transform(X_test)

In [30]:
X_train_sc.shape, y_train.shape

((2122, 86), (2122,))

### LinearRegression

In [31]:
lr = LinearRegression()
lr.fit(X_train, y_train)
print('Cross val score:', cross_val_score(lr, X_train, y_train))
print('lr train score:', lr.score(X_train, y_train)) # cross val score!!
print('lr test score:', lr.score(X_test, y_test))

Cross val score: [-6.38158893e+22  2.67276256e-01 -6.22182899e+20 -4.66960141e+22
  3.78753638e-01]
lr train score: 0.3488238865506774
lr test score: -2.674717416830444e+21


### Kneighbor Regression

In [32]:
knn_r = KNeighborsRegressor()

In [33]:
knn_r.fit(X_train_sc, y_train)
print('Cross val score:', cross_val_score(knn_r, X_train, y_train))
print('knn_r train score:', knn_r.score(X_train_sc, y_train)) # cross val score!!
print('knn_r test score:', knn_r.score(X_test_sc, y_test))

Cross val score: [0.4522765  0.20763932 0.14795488 0.42241846 0.02073784]
knn_r train score: 0.5089117352495856
knn_r test score: 0.0859955358807949


### DecisionTree Regression

In [34]:
dtr = DecisionTreeRegressor()

In [35]:
dtr.fit(X_train, y_train)
print('Cross val score:', cross_val_score(dtr, X_train, y_train))
print('dtr train score:', dtr.score(X_train, y_train)) # cross val score!!
print('dtr test score:', dtr.score(X_test, y_test))

Cross val score: [ 0.36545146 -1.05865283  0.02846909  0.28324337 -0.45570717]
dtr train score: 0.9341957402579177
dtr test score: 0.0009099527041063205


### Bagging Regression

In [36]:
bagr = BaggingRegressor(random_state=42)

In [37]:
bagr.fit(X_train, y_train)
print('Cross val score:', cross_val_score(bagr, X_train, y_train))
print('bagr train score:', bagr.score(X_train, y_train)) # cross val score!!
print('bagr test score:', bagr.score(X_test, y_test))

Cross val score: [ 0.54171614 -0.00863132  0.11149548  0.42612904  0.17625062]
bagr train score: 0.8050885949377834
bagr test score: 0.06838286250907155


### RandomForest Regression

In [38]:
rfr = RandomForestRegressor(random_state=42)

In [39]:
rfr.fit(X_train, y_train)
print('Cross val score:', cross_val_score(rfr, X_train, y_train))
print('rfr train score:', rfr.score(X_train, y_train)) # cross val score!!
print('rfr test score:', rfr.score(X_test, y_test))

Cross val score: [0.52923026 0.14390017 0.13166174 0.46360817 0.27202755]
rfr train score: 0.8343909747559816
rfr test score: 0.07793045961721035


### Adaboost Regression

In [40]:
abr = AdaBoostRegressor(random_state=42)

In [41]:
abr.fit(X_train, y_train)
print('Cross val score:', cross_val_score(abr, X_train, y_train))
print('abr train score:', abr.score(X_train, y_train)) # cross val score!!
print('abr test score:', abr.score(X_test, y_test))

Cross val score: [-0.42864036 -0.63215918  0.09066109 -0.31473327 -1.0430231 ]
abr train score: -1.1184199755355957
abr test score: -0.30399577011358114


### Regression SVC

In [42]:
svr = SVR()

In [43]:
%time svr.fit(X_train_sc, y_train)
print('Cross val score:', cross_val_score(svr, X_train_sc, y_train))
print('svr train score:', svr.score(X_train_sc, y_train)) # cross val score!!
print('svr test score:', svr.score(X_test_sc, y_test))

CPU times: user 509 ms, sys: 19.1 ms, total: 528 ms
Wall time: 543 ms
Cross val score: [0.07408135 0.11772983 0.0078114  0.04071852 0.14548772]
svr train score: 0.06469407735560084
svr test score: 0.0010115759579437666
